# Ridge and Lasso Regression - Lab

## Introduction

In this lab, you'll practice your knowledge on Ridge and Lasso regression!

## Objectives

You will be able to:

- Use Lasso and ridge regression in Python
- Compare Lasso and Ridge with standard regression

## Housing Prices Data

Let's look at yet another house pricing data set.

In [46]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('Housing_Prices/train.csv')

Look at df.info

In [2]:
# Your code here
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

We'll make a first selection of the data by removing some of the data with `dtype = object`, this way our first model only contains **continuous features**

Make sure to remove the SalesPrice column from the predictors (which you store in `X`), then replace missing inputs by the median per feature.

Store the target in `y`.

In [3]:
# remove "object"-type features from `X`
X = df.drop(df.loc[:, df.dtypes == object], axis=1)

In [4]:
# Create y
y = df.SalePrice

In [5]:
# remove SalePrice from `X`
X.drop('SalePrice', axis=1, inplace=True)

In [16]:
import numpy as np
for col in X:
    med = X[col].median()
    X[col].fillna(value = med, inplace = True)

Look at the information of `X` again

In [20]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 37 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
MasVnrArea       1460 non-null float64
BsmtFinSF1       1460 non-null int64
BsmtFinSF2       1460 non-null int64
BsmtUnfSF        1460 non-null int64
TotalBsmtSF      1460 non-null int64
1stFlrSF         1460 non-null int64
2ndFlrSF         1460 non-null int64
LowQualFinSF     1460 non-null int64
GrLivArea        1460 non-null int64
BsmtFullBath     1460 non-null int64
BsmtHalfBath     1460 non-null int64
FullBath         1460 non-null int64
HalfBath         1460 non-null int64
BedroomAbvGr     1460 non-null int64
KitchenAbvGr     1460 non-null int64
TotRmsAbvGrd     1460 non-null int64
F

## Let's use this data to perform a first naive linear regression model

Compute the R squared and the MSE for both train and test set.

In [32]:
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Perform train test split
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Fit the model and print R2 and MSE for train and test
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Training set
print("Training set")
print("R squared: ", linreg.score(X_train, y_train))
print("MSE: ", mean_squared_error(y_train, linreg.predict(X_train)))

# Test set
print("\nTest set")
print("R squared: ", linreg.score(X_test, y_test))
print("MSE: ", mean_squared_error(y_test, linreg.predict(X_test)))

Training set
R squared:  0.8465947900589366
MSE:  934139244.8207232

Test set
R squared:  0.6531328657190205
MSE:  2407888581.374319


## Normalize your data

We haven't normalized our data, let's create a new model that uses `preprocessing.scale` to scale our predictors!

In [42]:
from sklearn import preprocessing

# Scale the data and perform train test split
X_scaled = preprocessing.scale(X)
X_train, X_test, y_train, y_test = train_test_split(X_scaled,y)

Perform the same linear regression on this data and print out R-squared and MSE.

In [45]:
# Fit the model and print R2 and MSE for train and test
linreg_norm = LinearRegression()
linreg_norm.fit(X_train, y_train)

# Training set
print("Training set")
print("R squared: ", linreg_norm.score(X_train, y_train))
print("MSE: ", mean_squared_error(y_train, linreg_norm.predict(X_train)))

# Test set
print("\nTest set")
print("R squared: ", linreg_norm.score(X_test, y_test))
print("MSE: ", mean_squared_error(y_test, linreg_norm.predict(X_test)))

Training set
R squared:  0.8361732060739483
MSE:  1073071533.5639626

Test set
R squared:  0.683097093860708
MSE:  1763188208.4750044


## Include dummy variables

We haven't included dummy variables so far: let's use our "object" variables again and create dummies

In [47]:
# Create X_cat which contains only the categorical variables
X_cat = df.drop(df.loc[:, df.dtypes != object], axis=1)

In [51]:
# Make dummies
X_cat = pd.get_dummies(X_cat)

Merge `x_cat` together with our scaled `X` so you have one big predictor dataframe.

In [59]:
X_merged = pd.concat([pd.DataFrame(X_scaled), X_cat], axis = 1)

Perform the same linear regression on this data and print out R-squared and MSE.

In [60]:
# Perform train test split
X_train, X_test, y_train, y_test = train_test_split(X_merged, y)

# Fit the model and print R2 and MSE for train and test
linreg = LinearRegression()
linreg.fit(X_train, y_train)

# Training set
print("Training set")
print("R squared: ", linreg.score(X_train, y_train))
print("MSE: ", mean_squared_error(y_train, linreg.predict(X_train)))

# Test set
print("\nTest set")
print("R squared: ", linreg.score(X_test, y_test))
print("MSE: ", mean_squared_error(y_test, linreg.predict(X_test)))

Training set
R squared:  0.9101986592619868
MSE:  608643888.916895

Test set
R squared:  -4.119807445350674e+20
MSE:  2.0152777171055118e+30


Notice the severe overfitting above; our training R squared is quite high, but the testing R squared is negative! Our predictions are far far off. Similarly, the scale of the Testing MSE is orders of magnitude higher then that of the training.

## Perform Ridge and Lasso regression

Use all the data (normalized features and dummy categorical variables) and perform Lasso and Ridge regression for both! Each time, look at R-squared and MSE.

## Lasso

With default parameter (alpha = 1)

In [66]:
from sklearn.linear_model import Lasso, Ridge

In [63]:
lasso = Lasso(alpha=1)
lasso.fit(X_train, y_train)

# Training set
print("Training set")
print("R squared: ", lasso.score(X_train, y_train))
print("MSE: ", mean_squared_error(y_train, lasso.predict(X_train)))

# Test set
print("\nTest set")
print("R squared: ", lasso.score(X_test, y_test))
print("MSE: ", mean_squared_error(y_test, lasso.predict(X_test)))

Training set
R squared:  0.9474173601436512
MSE:  356387801.65937144

Test set
R squared:  0.7350127566504445
MSE:  1296232636.8972313


With a higher regularization parameter (alpha = 10)

In [64]:
lasso = Lasso(alpha=10)
lasso.fit(X_train, y_train)

# Training set
print("Training set")
print("R squared: ", lasso.score(X_train, y_train))
print("MSE: ", mean_squared_error(y_train, lasso.predict(X_train)))

# Test set
print("\nTest set")
print("R squared: ", lasso.score(X_test, y_test))
print("MSE: ", mean_squared_error(y_test, lasso.predict(X_test)))

Training set
R squared:  0.94568037640933
MSE:  368160504.8991473

Test set
R squared:  0.7558378468223289
MSE:  1194362972.5092165


## Ridge

With default parameter (alpha = 1)

In [70]:
ridge = Ridge(alpha=1)
ridge.fit(X_train, y_train)

# Training set
print("Training set")
print("R squared: ", ridge.score(X_train, y_train))
print("MSE: ", mean_squared_error(y_train, ridge.predict(X_train)))

# Test set
print("\nTest set")
print("R squared: ", ridge.score(X_test, y_test))
print("MSE: ", mean_squared_error(y_test, ridge.predict(X_test)))

Training set
R squared:  0.9329730912091221
MSE:  454286295.65314597

Test set
R squared:  0.7823273252558839
MSE:  1064784936.8048162


With default parameter (alpha = 10)

In [69]:
ridge = Ridge(alpha=10)
ridge.fit(X_train, y_train)

# Training set
print("Training set")
print("R squared: ", ridge.score(X_train, y_train))
print("MSE: ", mean_squared_error(y_train, ridge.predict(X_train)))

# Test set
print("\nTest set")
print("R squared: ", ridge.score(X_test, y_test))
print("MSE: ", mean_squared_error(y_test, ridge.predict(X_test)))

Training set
R squared:  0.9100063939562106
MSE:  609946999.7886215

Test set
R squared:  0.8063612643448058
MSE:  947218612.3038113


## Look at the metrics, what are your main conclusions?

Both lasso and ridge regression perform more consistently between training and test sets compared to linear regression without regularisation. There are small differences in results between alpha levels.

## Compare number of parameter estimates that are (very close to) 0 for Ridge and Lasso

In [76]:
# number of Ridge params almost zero
(abs(ridge.coef_) < 0.1).sum()

2

In [78]:
# number of Lasso params almost zero
(abs(lasso.coef_) < 0.1).sum()

58

Compare with the total length of the parameter space and draw conclusions!

In [79]:
len(lasso.coef_)

289

Lasso also performed feature elimination, eliminating about 20% of features while still retaining a good r squared and MSE.

## Summary

Great! You now know how to perform Lasso and Ridge regression.